In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
advisories_df = pd.read_csv('../../data/processed/Travel_Advisories_Processed.csv')
gpi_df = pd.read_csv('../../data/processed/Global_Peace_Index_Processed.csv')
terror_df = pd.read_csv('../../data/processed/Global_Terrorism_Index_Processed.csv')
protest_df = pd.read_csv('../../data/processed/Global_Protest_Processed.csv')
deaths_df = pd.read_csv('../../data/processed/us_citizen_deaths_overseas_processed.csv')

In [3]:
# Filter to 2024 advisories only 
advisories_df['Date Updated'] = pd.to_datetime(advisories_df['Date Updated'], errors='coerce')
advisories_2024 = advisories_df[advisories_df['Date Updated'].dt.year == 2024]

In [4]:
# Global Peace Index: lower is better, use 2024 score
gpi_df = gpi_df.rename(columns={'country': 'Country', '2024_Score': 'Global Peace Index'})[['Country', 'Global Peace Index']]

# Global Terrorism Index: higher means more terrorism
terror_df = terror_df.rename(columns={'country': 'Country', '2024 score': 'Terrorism Score'})[['Country', 'Terrorism Score']]

# Protest data: aggregate number of protests per country
protest_df = protest_df.rename(columns={'Country': 'Country'})  # Already fine
protest_agg = protest_df.groupby('Country').size().reset_index(name='Protest Score')

In [5]:
# US deaths abroad: count number of US citizen deaths per country
deaths_df = deaths_df.rename(columns={'country': 'Country'})
death_agg = deaths_df.groupby('Country').size().reset_index(name='Death Count')

In [6]:
merged_df = advisories_2024.merge(gpi_df, on='Country', how='left') \
    .merge(terror_df, on='Country', how='left') \
    .merge(protest_agg, on='Country', how='left') \
    .merge(death_agg, on='Country', how='left')

In [7]:
# Drop countries with any missing data
merged_df = merged_df.dropna(subset=['Global Peace Index', 'Terrorism Score', 'Protest Score', 'Death Count'])

In [8]:
# Normalize each risk metric to [0, 1]
# This allows averaging them into a composite risk score
scaler = MinMaxScaler()
risk_features = ['Global Peace Index', 'Terrorism Score', 'Protest Score', 'Death Count']
merged_df[risk_features] = scaler.fit_transform(merged_df[risk_features])

In [9]:
# Average of all normalized risk indicators
merged_df['Composite Risk Score'] = merged_df[risk_features].mean(axis=1)

# Normalize the US advisory level (1–4) to a 0.25–1.0 scale
merged_df['Normalized Advisory'] = merged_df['Advisory Level Number'] / 4.0

# Alignment Score = absolute difference between risk and advisory level
merged_df['Alignment Score'] = (merged_df['Normalized Advisory'] - merged_df['Composite Risk Score']).abs()

In [10]:
top_aligned = merged_df.nsmallest(5, 'Alignment Score')
top_misaligned = merged_df.nlargest(5, 'Alignment Score')

In [11]:
top_aligned 

,Country,Advisory Level,Date Updated,Advisory Level Number,Global Peace Index,Terrorism Score,Protest Score,Death Count,Composite Risk Score,Normalized Advisory,Alignment Score
40,Poland,Level 1: Exercise Normal Precautions,2024-05-01,1.0,0.185562,0.257683,0.518519,0.015899,0.244416,0.25,0.005584
31,Greece,Level 1: Exercise Normal Precautions,2024-08-15,1.0,0.244026,0.384555,0.444444,0.039417,0.278111,0.25,0.028111
122,Cameroon,Level 2: Exercise Increased Caution,2024-12-18,2.0,0.742247,0.912004,0.148148,0.002319,0.451180,0.50,0.048820
114,Argentina,Level 1: Exercise Normal Precautions,2024-09-20,1.0,0.275547,0.105201,0.370370,0.012918,0.191009,0.25,0.058991
92,Senegal,Level 1: Exercise Normal Precautions,2024-10-21,1.0,0.391967,0.207250,0.148148,0.001987,0.187338,0.25,0.062662


In [12]:
top_misaligned

,Country,Advisory Level,Date Updated,Advisory Level Number,Global Peace Index,Terrorism Score,Protest Score,Death Count,Composite Risk Score,Normalized Advisory,Alignment Score
24,Belarus,Level 4: Do Not Travel,2024-12-18,4.0,0.497204,0.030602,0.111111,0.000000,0.159729,1.0,0.840271
74,Libya,Level 4: Do Not Travel,2024-08-01,4.0,0.617692,0.211715,0.074074,0.000331,0.225953,1.0,0.774047
73,Lebanon,Level 4: Do Not Travel,2024-12-27,4.0,0.701576,0.162464,0.111111,0.007618,0.245692,1.0,0.754308
110,Venezuela,Level 4: Do Not Travel,2024-09-24,4.0,0.766650,0.093249,0.148148,0.013581,0.255407,1.0,0.744593
63,Haiti,Level 4: Do Not Travel,2024-09-18,4.0,0.769700,0.000000,0.296296,0.079828,0.286456,1.0,0.713544


To evaluate how well U.S. travel advisories align with real-world risk, we combined data from four global indicators: the Global Peace Index, Global Terrorism Index, protest activity, and U.S. citizen deaths overseas. For each country, we first calculated the number of active protests and the total number of U.S. citizen deaths. Each of these four risk indicators was then normalized to a 0–1 scale using Min-Max normalization, ensuring comparability regardless of original units. We averaged the normalized values to create a Composite Risk Score, where a higher score indicates greater overall risk. U.S. travel advisories were converted to numeric levels (1 to 4) and then normalized by dividing by 4, resulting in a Normalized Advisory Score on the same 0–1 scale. Finally, the Alignment Score was calculated as the absolute difference between the Composite Risk Score and the Normalized Advisory Score. A low alignment score means the advisory closely matches the risk level, while a high score indicates potential misalignment.

This analysis relies on the availability and quality of external data sources, which may vary in coverage, accuracy, or timeliness. For instance, protest counts or terrorism scores may underreport events in certain regions. Also, the equal weighting of all four indicators assumes they contribute equally to travel risk, which may not reflect how the U.S. government prioritizes threats. Additionally, some U.S. advisories are based on diplomatic or political concerns that aren't captured in quantitative risk metrics, so alignment may not always indicate correctness.

Despite these limitations, the graph provides a valuable data-driven lens into how U.S. travel advisories compare with globally recognized indicators of risk. By normalizing and aggregating independent sources, we can consistently compare countries and identify patterns or outliers. This helps spotlight where the U.S. might be over- or under-advising, fostering transparency and critical discussion about how travel warnings are issued.


In [18]:
# Aligned Countries Plot
fig_aligned = px.bar(
    top_aligned,
    x='Alignment Score',
    y='Country',
    orientation='h',
    title='Top 5 Countries with Most Aligned US Travel Advisories',
    labels={'Alignment Score': 'Alignment Score (Lower is Better)', 'Country': 'Country'},
)
fig_aligned.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    plot_bgcolor='white',
    paper_bgcolor='white'
)
aligned_path = '../../img/top_5_aligned_advisories.html'
fig_aligned.write_html(aligned_path)
fig_aligned.show()

# Misaligned Countries Plot
fig_misaligned = px.bar(
    top_misaligned,
    x='Alignment Score',
    y='Country',
    orientation='h',
    title='Top 5 Countries with Most Misaligned US Travel Advisories',
    labels={'Alignment Score': 'Alignment Score (Higher is Worse)', 'Country': 'Country'},
)
fig_misaligned.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    plot_bgcolor='white',
    paper_bgcolor='white'
)
misaligned_path = '../../img/top_5_misaligned_advisories.html'
fig_misaligned.write_html(misaligned_path)
fig_misaligned.show()
